In [1]:
from keras.models import load_model

import matplotlib.pyplot as plt
import numpy as np
import math
import copy
from PIL import Image

from keras.models import Sequential, Model
from keras.layers import Input, Dense, Flatten, Dropout, Activation, Lambda, Permute, Reshape
from keras.layers import Convolution1D, Convolution2D, ZeroPadding2D, MaxPooling2D

import os

Using TensorFlow backend.


In [2]:
trial_model=load_model('Utilities\\VGGFace.h5')

def features(featmodel, crpimg, transform=False): #extract face feature vector
    
    # transform=True seems more robust but I think the RGB channels are not in right order
    
    imarr = np.array(crpimg).astype(np.float32)

    if transform:
        imarr[:,:,0] -= 129.1863
        imarr[:,:,1] -= 104.7624
        imarr[:,:,2] -= 93.5940        
        aux = copy.copy(imarr)
    imarr = np.expand_dims(imarr, axis=0)
    output= featmodel.predict(imarr)[0,:]
    return output


C:\Users\Om\Anaconda3\envs\opencv\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [3]:
def feature_extractor(inpDir,personID,wordID,timestep): #access directory and extract face vector
    
    
        
        
    sample_space=[] #for appending different utterances
    feature_sequence = [] # for appending different timesteps
    inpDir=inpDir #input dataset
    linpDir = os.listdir(inpDir) #list all directories in dataset
    #print(linpDir)
    personStr= linpDir[personID]
    personFolder = '%s\\%s\\words' % (inpDir,personStr) #opening words folder
    lpersonFolder = os.listdir(personFolder)
    #print(lpersonFolder)
    wordID= lpersonFolder[wordID]
    #print(wordID)
    wordFolder = '%s\\%s' % (personFolder,wordID)
    lwordFolder = os.listdir(wordFolder)
    #print(lwordFolder)
    
    i=0
    for utterance in lwordFolder:
        
        utterFolder= '%s\\%s' % (wordFolder,utterance)
        lutterFolder = os.listdir(utterFolder)
        colourFolder ='%s\\%s' % (utterFolder,lutterFolder[0])
        lcolourFolder = os.listdir(colourFolder)
        frame = lcolourFolder[timestep]
        i += 1
        image= "%s\\%s" % (colourFolder,frame) 
        im=Image.open(image)
        im = im.resize((224,224))
        feature_vector = features(trial_model,im, transform=True).reshape((1,1, 1, 2622))
        if i==1 :
            feature_sequence=feature_vector #done because of need of same dim. for concatenation
        else:
            feature_sequence = np.concatenate((feature_sequence,feature_vector),axis=0)
        #print(feature_sequence.shape)
            
    feature_sequence_reshaped = feature_sequence[:,0,:,:] #removing unnecessary extra dimensions
    return feature_sequence_reshaped


In [4]:
#test1=feature_extractor(r'C:\Users\Om\Desktop\dataset',0,0,0)

In [5]:
def word_extractor(inpDir,person,sample):
    timesteps=20 #optimum number of timesteps.Change can be made.
    condition_count=0 #
    for i in range(timesteps):
        CNN_out=feature_extractor(inpDir,person,sample,i)#calling feature_extractor()
        condition_count = condition_count + 1
        if condition_count==1 :
            CNN_out_total= CNN_out # extra condtion is added to maintain same dimensions
        else :
            CNN_out_total= np.concatenate((CNN_out_total,CNN_out))
        print(condition_count)
    return CNN_out_total 

In [6]:
#CNN_out_total= word_extractor(r'C:\Users\Om\Desktop\dataset',0,0)

In [7]:
def person_extractor(InpDir,person):
    p_dataset=[] #array to store values
    condition_count=0
    no_of_words= 7 
    for i in range(no_of_words):
        print(str(i)+'word')
        condition_count = condition_count +1
        CNN_out_total= word_extractor(InpDir,person,i)
        if condition_count==1 :
            p_dataset = CNN_out_total #done because of need of same dimensions for concatenation
        else:
            p_dataset = np.concatenate((p_dataset,CNN_out_total),axis=0)
        
    return p_dataset 

In [8]:
def reshape_LSTM(sample_mat):
    feature_vector=[]
    feature_mat=[]
    
    count_word = 0
    for word in range(3):
        count_i=0
        for utterance in range(10):
            count_j=0
            for timestep in range(200*word,200*(word+1),10):
                timestep = timestep + utterance
                sample_mat_j=sample_mat[timestep]
                sample_mat_j=sample_mat_j.reshape((1,2622))
                if count_j==0:
                    feature_vector=sample_mat_j
                else:
                    feature_vector=np.concatenate((feature_vector,sample_mat_j))
                count_j = count_j + 1

            if count_i==0:
                feature_mat=feature_vector
            else:
                feature_mat= np.concatenate((feature_mat,feature_vector))
            count_i= count_i + 1
            print(feature_mat.shape)
            
        if count_word==0:
            LSTM_input=feature_mat
        else:
            LSTM_input= np.concatenate((LSTM_input,feature_mat))
        count_word= count_word + 1
                
    print(LSTM_input.shape)
  
    return LSTM_input

In [9]:
def BottleNeck(InpDir,person):
    p_dataset=person_extractorT(InpDir,person)
    p_dataset_LSTM=reshape_LSTM(p_dataset)
    p_total_dataset=p_dataset_LSTM
    print('person done')
    return p_total_dataset

In [10]:
#p_total_dataset_1=BottleNeck(r'C:\Users\Om\Desktop\dataset',0)

In [11]:
from numpy import save

In [12]:
#np.save('F01.npy',p_total_dataset_1)

In [13]:
#p_total_dataset_2=BottleNeck(r'C:\Users\Om\Desktop\dataset',1)

In [14]:
#np.save('F02.npy',p_total_dataset_2)

In [15]:
#p_total_dataset_3=BottleNeck(r'C:\Users\Om\Desktop\dataset',2)

In [16]:
#np.save('F04.npy',p_total_dataset_3)


In [17]:
#p_total_dataset_4=BottleNeck(r'C:\Users\Om\Desktop\dataset',3)

In [18]:
#np.save('F05.npy',p_total_dataset_4)

In [19]:
#p_total_dataset_5=BottleNeck(r'C:\Users\Om\Desktop\dataset',4)

In [20]:
#p_total_dataset_6=BottleNeck(r'C:\Users\Om\Desktop\dataset',5)

In [21]:
#np.save('F07.npy',p_total_dataset_6)

In [22]:
def person_extractorT(InpDir,person):
    p_dataset=[] #array to store values
    condition_count=0
    no_of_words= 3 
    for i in range(3):
        print(str(i)+'word')
        condition_count = condition_count +1
        CNN_out_total= word_extractor(InpDir,person,i)
        if condition_count==1 :
            p_dataset = CNN_out_total #done because of need of same dimensions for concatenation
        else:
            p_dataset = np.concatenate((p_dataset,CNN_out_total),axis=0)
        
    return p_dataset

In [23]:
#p_test1=BottleNeck(r'C:\Users\Om\Desktop\dataset',0)

In [24]:
#np.save('F01_t.npy',p_test1)

In [25]:
#p_test2=BottleNeck(r'C:\Users\Om\Desktop\dataset',1)
#np.save('F02_t.npy',p_test2)

In [26]:
#p_test3=BottleNeck(r'C:\Users\Om\Desktop\dataset',2)
#np.save('F04_t.npy',p_test3)

In [27]:
#p_test4=BottleNeck(r'C:\Users\Om\Desktop\dataset',3)
#np.save('F05_t.npy',p_test4)

In [28]:
#p_test5=BottleNeck(r'C:\Users\Om\Desktop\dataset',4)
#np.save('F06_t.npy',p_test5)

In [29]:
p_test5=BottleNeck(r'C:\Users\Om\Desktop\dataset',5)
np.save('F07_t.npy',p_test5)

0word
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1word
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
2word
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
(20, 2622)
(40, 2622)
(60, 2622)
(80, 2622)
(100, 2622)
(120, 2622)
(140, 2622)
(160, 2622)
(180, 2622)
(200, 2622)
(20, 2622)
(40, 2622)
(60, 2622)
(80, 2622)
(100, 2622)
(120, 2622)
(140, 2622)
(160, 2622)
(180, 2622)
(200, 2622)
(20, 2622)
(40, 2622)
(60, 2622)
(80, 2622)
(100, 2622)
(120, 2622)
(140, 2622)
(160, 2622)
(180, 2622)
(200, 2622)
(600, 2622)
person done


In [30]:
p_test4=BottleNeck(r'C:\Users\Om\Desktop\dataset',6)
np.save('F08_t.npy',p_test4)

0word
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1word
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
2word
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
(20, 2622)
(40, 2622)
(60, 2622)
(80, 2622)
(100, 2622)
(120, 2622)
(140, 2622)
(160, 2622)
(180, 2622)
(200, 2622)
(20, 2622)
(40, 2622)
(60, 2622)
(80, 2622)
(100, 2622)
(120, 2622)
(140, 2622)
(160, 2622)
(180, 2622)
(200, 2622)
(20, 2622)
(40, 2622)
(60, 2622)
(80, 2622)
(100, 2622)
(120, 2622)
(140, 2622)
(160, 2622)
(180, 2622)
(200, 2622)
(600, 2622)
person done


In [31]:
p_test3=BottleNeck(r'C:\Users\Om\Desktop\dataset',7)
np.save('F09_t.npy',p_test3)

0word
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1word
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
2word
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
(20, 2622)
(40, 2622)
(60, 2622)
(80, 2622)
(100, 2622)
(120, 2622)
(140, 2622)
(160, 2622)
(180, 2622)
(200, 2622)
(20, 2622)
(40, 2622)
(60, 2622)
(80, 2622)
(100, 2622)
(120, 2622)
(140, 2622)
(160, 2622)
(180, 2622)
(200, 2622)
(20, 2622)
(40, 2622)
(60, 2622)
(80, 2622)
(100, 2622)
(120, 2622)
(140, 2622)
(160, 2622)
(180, 2622)
(200, 2622)
(600, 2622)
person done


In [32]:
p_test2=BottleNeck(r'C:\Users\Om\Desktop\dataset',8)
np.save('F10_t.npy',p_test2)

0word
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1word
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
2word
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
(20, 2622)
(40, 2622)
(60, 2622)
(80, 2622)
(100, 2622)
(120, 2622)
(140, 2622)
(160, 2622)
(180, 2622)
(200, 2622)
(20, 2622)
(40, 2622)
(60, 2622)
(80, 2622)
(100, 2622)
(120, 2622)
(140, 2622)
(160, 2622)
(180, 2622)
(200, 2622)
(20, 2622)
(40, 2622)
(60, 2622)
(80, 2622)
(100, 2622)
(120, 2622)
(140, 2622)
(160, 2622)
(180, 2622)
(200, 2622)
(600, 2622)
person done


In [33]:
p_test1=BottleNeck(r'C:\Users\Om\Desktop\dataset',9)
np.save('F11_t.npy',p_test1)

0word
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1word
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
2word
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
(20, 2622)
(40, 2622)
(60, 2622)
(80, 2622)
(100, 2622)
(120, 2622)
(140, 2622)
(160, 2622)
(180, 2622)
(200, 2622)
(20, 2622)
(40, 2622)
(60, 2622)
(80, 2622)
(100, 2622)
(120, 2622)
(140, 2622)
(160, 2622)
(180, 2622)
(200, 2622)
(20, 2622)
(40, 2622)
(60, 2622)
(80, 2622)
(100, 2622)
(120, 2622)
(140, 2622)
(160, 2622)
(180, 2622)
(200, 2622)
(600, 2622)
person done
